In [1]:
%cd ..

/home/lingze/embedding_fusion


In [2]:
import numpy as np
import pandas as pd
from relbench.datasets import get_dataset
from relbench.base import Table
from tqdm import tqdm
from typing import Any,Dict

In [3]:
import torch
import os
from torch import Tensor
from torch_frame import stype
from torch_frame.config import TextEmbedderConfig
from torch_frame.data import Dataset
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.typing import NodeType
from torch_geometric.utils import sort_edge_index

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = get_dataset(name = "rel-avito", download = True)
db = dataset.get_db()

Loading Database object from /home/lingze/.cache/relbench/rel-avito/db...
Done in 4.74 seconds.


In [5]:
from utils.preprocess import infer_type_in_db
col_type_dict = infer_type_in_db(db, True)

[rule 0]: PhoneRequestsStream Inferred UserID from numerical as categorical
[rule 0]: PhoneRequestsStream Inferred IPID from numerical as categorical
[rule 0]: PhoneRequestsStream Inferred AdID from numerical as categorical
[rule 0]: Location Inferred LocationID from numerical as categorical
[rule 1]: Location Inferred Level from numerical as categorical
[rule 0]: Location Inferred RegionID from numerical as categorical
[rule 0]: Location Inferred CityID from numerical as categorical
[rule 0]: SearchInfo Inferred UserID from numerical as categorical
[rule 0]: SearchInfo Inferred SearchID from numerical as categorical
[rule 0]: SearchInfo Inferred IPID from numerical as categorical
[rule 1]: SearchInfo Inferred IsUserLoggedOn from numerical as categorical
[rule 0]: SearchInfo Inferred LocationID from numerical as categorical
[rule 0]: UserInfo Inferred UserID from numerical as categorical
[rule 0]: UserInfo Inferred UserAgentID from numerical as categorical
[rule 1]: UserInfo Inferred U

In [6]:
for table_name, table in db.table_dict.items():
    print(f'table_name: {table_name}, pkey:{table.pkey_col}')
    for fk in table.fkey_col_to_pkey_table.keys():
        print(f'tablename: {table_name}, fk: {fk}')


table_name: PhoneRequestsStream, pkey:None
tablename: PhoneRequestsStream, fk: UserID
tablename: PhoneRequestsStream, fk: AdID
table_name: Location, pkey:LocationID
table_name: SearchInfo, pkey:SearchID
tablename: SearchInfo, fk: UserID
tablename: SearchInfo, fk: LocationID
tablename: SearchInfo, fk: CategoryID
table_name: UserInfo, pkey:UserID
table_name: SearchStream, pkey:None
tablename: SearchStream, fk: SearchID
tablename: SearchStream, fk: AdID
table_name: VisitStream, pkey:None
tablename: VisitStream, fk: UserID
tablename: VisitStream, fk: AdID
table_name: AdsInfo, pkey:AdID
tablename: AdsInfo, fk: LocationID
tablename: AdsInfo, fk: CategoryID
table_name: Category, pkey:CategoryID


In [7]:
col_type_dict

{'PhoneRequestsStream': {'UserID': <stype.categorical: 'categorical'>,
  'IPID': <stype.categorical: 'categorical'>,
  'AdID': <stype.categorical: 'categorical'>,
  'PhoneRequestDate': <stype.timestamp: 'timestamp'>},
 'Location': {'LocationID': <stype.categorical: 'categorical'>,
  'Level': <stype.categorical: 'categorical'>,
  'RegionID': <stype.categorical: 'categorical'>,
  'CityID': <stype.categorical: 'categorical'>},
 'SearchInfo': {'UserID': <stype.categorical: 'categorical'>,
  'SearchID': <stype.categorical: 'categorical'>,
  'SearchDate': <stype.timestamp: 'timestamp'>,
  'IPID': <stype.categorical: 'categorical'>,
  'IsUserLoggedOn': <stype.categorical: 'categorical'>,
  'SearchQuery': <stype.text_embedded: 'text_embedded'>,
  'LocationID': <stype.categorical: 'categorical'>,
  'CategoryID': <stype.categorical: 'categorical'>},
 'UserInfo': {'UserID': <stype.categorical: 'categorical'>,
  'UserAgentID': <stype.categorical: 'categorical'>,
  'UserAgentOSID': <stype.categoric

In [8]:
# print text embedding type, profile
for table, type_dict in col_type_dict.items():
    for col_name, stype in type_dict.items():
        if stype == stype.text_embedded:
            unique_value = db.table_dict[table].df[col_name].unique()
            n = len(unique_value)
            nm = len(db.table_dict[table].df)
            nan_num = db.table_dict[table].df[col_name].isnull().sum()
            print(f"{table}.{col_name}: {stype}, unique values: {n}/{nm} Nan Value: {nan_num}/{nm}")

SearchInfo.SearchQuery: text_embedded, unique values: 13210/1987156 Nan Value: 1682279/1987156
AdsInfo.Title: text_embedded, unique values: 3246389/5960558 Nan Value: 0/5960558


In [9]:
from typing import List, Optional
from torch_frame.config.text_embedder import TextEmbedderConfig
from sentence_transformers import SentenceTransformer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device
                                       ] = None):
        self.model = SentenceTransformer(
            "all-MiniLM-L12-v2",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=512
)

def remove_pkey_fkey(col_to_stype: Dict[str, Any], table:Table) -> dict:
    r"""Remove pkey, fkey columns since they will not be used as input feature."""
    if table.pkey_col is not None:
        if table.pkey_col in col_to_stype:
            col_to_stype.pop(table.pkey_col)
    for fkey in table.fkey_col_to_pkey_table.keys():
        if fkey in col_to_stype:
            col_to_stype.pop(fkey)

def to_unix_time(ser: pd.Series) -> np.ndarray:
    r"""Converts a :class:`pandas.Timestamp` series to UNIX timestamp (in seconds)."""
    assert ser.dtype in [np.dtype("datetime64[s]"), np.dtype("datetime64[ns]")]
    unix_time = ser.astype("int64").values
    if ser.dtype == np.dtype("datetime64[ns]"):
        unix_time //= 10**9
    return unix_time

In [10]:
# preprocess the table, concatenate the columns which is text type
#        /--- text_col_1 ---/ --- text_col_2 --- / --- text_col_3 --- / 
# row 1  /------- A   -----/ ------- B   -----  / -----   C  ------- /
# -------> Generate a new TexT column
# "text_col_1 is A, text_col_2 is B, text_col_3 is C"

# Therefore, we only need to convert this text column to vector 
# and drop the original text columns
# for saving memory and computation 

for table_name, type_dict in col_type_dict.items():
    # collect the text columns
    text_cols = [ col for col, stype in type_dict.items() if stype == stype.text_embedded]
    compress_cols = []
    # for long text, we still keep it as one column
    for col in text_cols:
        avg_word_count = db.table_dict[table_name].df[col].dropna().apply(lambda x: len(str(x).split())).mean()
        if avg_word_count < 128: # a half of default max length of BERT Max length （256）
            # remove the long text cols
            compress_cols.append(col)
          
    
    if len(compress_cols) <= 1:
        # if only one text column, we do not need to compress
        continue
    
    print(f"----> Compressing {table_name} text columns: {compress_cols}")
    
    df = db.table_dict[table_name].df
    compress_text_df = df[compress_cols]
    
    def row_to_text(row):
        if row.isna().all():
            return None
        tokens = [f"{key} is {value}" for key, value in row.dropna().items()]
        return ", ".join(tokens)

    text_list = compress_text_df.apply(row_to_text, axis=1).tolist()
    
    # drop the compressed columns
    df.drop(columns=compress_cols, inplace=True)
    df["text_compress"] = text_list
    
    # update the type dict
    for col in compress_cols:
        type_dict.pop(col)
    type_dict["text_compress"] = stype.text_embedded

In [9]:
# no multi-column text embedding

In [11]:
# start build graph
cache_dir = "./data/rel-avito-tensor-frame"
if cache_dir is not None:
    os.makedirs(cache_dir, exist_ok=True)
data = HeteroData()
col_stats_dict = {}
for table_name, table in db.table_dict.items():
    df = table.df
    # (important for foreignKey value) Ensure the pkey is consecutive
    if table.pkey_col is not None:
        assert (df[table.pkey_col].values == np.arange(len(df))).all()
    
    col_to_stype = col_type_dict[table_name]
    
    # remove pkey, fkey
    remove_pkey_fkey(col_to_stype, table)
    
    if len(col_to_stype) == 0:
        # for example, relationship table which only contains pkey and fkey
        raise KeyError(f"{table_name} has no column to build graph")
    
    path = (
            None if cache_dir is None else os.path.join(cache_dir, f"{table_name}.pt")
    )
    
    print(f"-----> Materialize {table_name} Tensor Frame")
    dataset = Dataset(
        df = df,
        col_to_stype=col_to_stype,
        col_to_text_embedder_cfg=text_embedder_cfg,
    ).materialize(path=path)
    
    data[table_name].tf = dataset.tensor_frame
    col_stats_dict[table_name] = dataset.col_stats
    
    # Add time attribute
    if table.time_col is not None:
        data[table_name].time = torch.from_numpy(
            to_unix_time(df[table.time_col])
        )

In [11]:
for table, type_dict in col_type_dict.items():
    for col_name, stype in type_dict.items():
        if stype == stype.text_embedded:
            unique_value = db.table_dict[table].df[col_name].unique()
            n = len(unique_value)
            nm = len(db.table_dict[table].df)
            nan_num = db.table_dict[table].df[col_name].isnull().sum()
            print(f"{table}.{col_name}: {stype} --> unique values: {n}/{nm} || Nan Value: {nan_num}/{nm}")
        

SearchInfo.SearchQuery: text_embedded --> unique values: 13210/1987156 || Nan Value: 1682279/1987156
AdsInfo.Title: text_embedded --> unique values: 3246389/5960558 || Nan Value: 0/5960558


In [12]:
# cache the col_type_dict
import pickle
with open(f"{cache_dir}/col_type_dict.pkl", "wb") as f:
    pickle.dump(col_type_dict, f)